In [1]:
import requests
import pandas as pd
from anytree import Node, RenderTree
import random
import string
from functions import *

In [2]:
with open('Authentication/database_uri.txt', 'r', encoding="utf8") as f:
    uri = f.read()

In [3]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

def connect_to_endpoint(url, headers, next_token = None):    
    response = requests.request("GET", url, headers = headers)
        
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

def getTweets(user_id, header):
    tweets_url = f'https://api.twitter.com/2/users/{user_id}/tweets'
    return connect_to_endpoint(tweets_url, header)

# 'conversation_id' is the identifier for the main tweet
def getConversation(conversation_id, max_results, header):
    params = 'in_reply_to_user_id,author_id,created_at,conversation_id'
    getConversation_url = f'https://api.twitter.com/2/tweets/search/recent?query=conversation_id:{conversation_id}&tweet.fields={params}&max_results={max_results}'

    return connect_to_endpoint(getConversation_url, header)

# For now we will return the time only
def getTweetInformation(conversation_id, header):
    params = 'created_at,conversation_id,in_reply_to_user_id,author_id,referenced_tweets'
    tweetInfo_url = f'https://api.twitter.com/2/tweets?tweet.fields={params}&ids={conversation_id}'
    
    result = connect_to_endpoint(tweetInfo_url, header)
    return result['data'][0]['created_at']

def getTweetComments(conversation_data):
    conversation_dict = {'id':[], 'timestamp':[], 'reply_to':[], 'comment':[]} #, 'new_id':[]
    
    for i in range(len(conversation_data['data'])):
        print('User ID:', conversation_data['data'][i]['author_id'], 
              'Time:', conversation_data['data'][i]['created_at'])
        print('In reply to:', conversation_data['data'][i]['in_reply_to_user_id'])
        print(conversation_data['data'][i]['text'], '\n')
        
        conversation_dict['id'].append(conversation_data['data'][i]['author_id'])
        conversation_dict['timestamp'].append(conversation_data['data'][i]['created_at'])
        conversation_dict['reply_to'].append(conversation_data['data'][i]['in_reply_to_user_id'])
        conversation_dict['comment'].append(conversation_data['data'][i]['text'])
#         conversation_dict['new_id'].append(generateRandomID())
        
    return conversation_dict

def generateRandomID(strlen = 8):
    return ''.join(random.choice(string.ascii_uppercase + string.ascii_lowercase + string.digits) for _ in range(strlen))

def getParentID(dataframe):
    parent_id = []
    root = generateRandomID()

    for index, row in dataframe.iterrows():
        if row['reply_to'] != '87818409':
            count = 0
            
            for _, i in dataframe.iterrows():
                if row['reply_to'] == i['id']:
                    parent_id.append(i['new_id'])
                    break
                
                count += 1

                if count == len(dataframe) + 1:
                    print('lol')
                    parent_id.append(i['nan'])
        
        else:
            parent_id.append(root)
            
    return parent_id

In [4]:
with open('Authentication/twitter_bearer_token.txt', 'r', encoding="utf8") as f:
    token = f.read()

header = create_headers(token)
max_results = 100

# The Straits Times
## Get Tweets

In [5]:
ST_id = '37874853'
ST_tweets = getTweets(ST_id, header)

In [6]:
for i in range(len(ST_tweets['data'])):
    print('Tweet ID:', ST_tweets['data'][i]['id'], 
          'Time:', getTweetInformation(ST_tweets['data'][i]['id'], header))
    print(ST_tweets['data'][i]['text'], '\n')
    
#     command = (
#             '''
#             INSERT INTO twitter_data
#             VALUES ('%s', '%s', '%s');
#             ''' % (ST_tweets['data'][i]['id'], getTweetInformation(ST_tweets['data'][i]['id'], header), 
#                    ST_tweets['data'][i]['text'])
#             )
#     setUpDB(command, uri)

Tweet ID: 1492447660216778754 Time: 2022-02-12T10:37:05.000Z
RT @STsportsdesk: Winter Olympics: China's Gao wins speed skating 500m gold in Games record time https://t.co/P9egledYac 

Tweet ID: 1492447653086777346 Time: 2022-02-12T10:37:03.000Z
RT @STForeignDesk: Protesters defy injunction order, continue to occupy key US-Canada bridge https://t.co/3ZRqo5Decz 

Tweet ID: 1492444251631742976 Time: 2022-02-12T10:23:32.000Z
This week's top reads from The Straits Times https://t.co/u6QKkYWOOe 

Tweet ID: 1492441253316743168 Time: 2022-02-12T10:11:37.000Z
RT @STsportsdesk: Sports academies, parents taking time to adjust to revised Covid-19 rules for children https://t.co/ggjRxiKzym 

Tweet ID: 1492441246639345669 Time: 2022-02-12T10:11:36.000Z
RT @STsportsdesk: Athletics: Olympic 100m champion Jacobs wins second 60m indoor race in a week https://t.co/ODWBUyn0hg 

Tweet ID: 1492441239764934659 Time: 2022-02-12T10:11:34.000Z
RT @ST_LifeTweets: Andrea DeCruz, Pierre Png celebrate 20th annivers

In [7]:
# query = '@straits_times'
# tweets_url = f'https://api.twitter.com/2/tweets/search/recent?query={query}&max_results={max_results}'
# connect_to_endpoint(tweets_url, header)

## Get Conversation

In [8]:
ST_conversation_data = getConversation('1492430537268416512', max_results, header)

In [9]:
ST_result = getTweetComments(ST_conversation_data)

User ID: 155512494 Time: 2022-02-12T10:37:33.000Z
In reply to: 37874853
@straits_times okay. 

User ID: 1250959902299656192 Time: 2022-02-12T10:37:12.000Z
In reply to: 37874853
@straits_times Any folks can see from UK, Israel that omicron will pump the daily cases. Their data also show the 3rd n 4th clot shot is not working. But @LawrenceWongST decided to overlook it and promote fear inorder to jab the people? With the help of state own media and fb kol ho chin 

User ID: 249757744 Time: 2022-02-12T09:59:00.000Z
In reply to: 297266496
@jsz2017 @straits_times Are you seriously that dumb? 

User ID: 297266496 Time: 2022-02-12T09:48:54.000Z
In reply to: 37874853
@straits_times if he said this before MOH started the jab coercing plan, then I could believe him😂 Did he said “even with jabs, there could be more than 10k cases every day?” No, right? Then anybody remember what he said by that time about how jabs would work? https://t.co/3rXclRadCq 

User ID: 567250256 Time: 2022-02-12T09:44:18.

In [10]:
ST_df = pd.DataFrame.from_dict(ST_result)
ST_df['id'] = ST_df['id'].astype(str)
ST_df['reply_to'] = ST_df['reply_to'].astype(str)
ST_df['url'] = ST_df['comment'].apply(lambda x: getLinks(x))
ST_df['link_title'] = ST_df['url'].apply(lambda x: getURLfromList(x))
ST_df.to_csv('Datasets/ST_twitter_data.csv', index=False)

ST_df.head()

,id,timestamp,reply_to,comment,url,link_title
0,155512494,2022-02-12T10:37:33.000Z,37874853,@straits_times okay.,,
1,1250959902299656192,2022-02-12T10:37:12.000Z,37874853,"@straits_times Any folks can see from UK, Isra...",,
2,249757744,2022-02-12T09:59:00.000Z,297266496,@jsz2017 @straits_times Are you seriously that...,,
3,297266496,2022-02-12T09:48:54.000Z,37874853,@straits_times if he said this before MOH star...,https://twitter.com/lukewearechange/status/147...,[twitter.com]
4,567250256,2022-02-12T09:44:18.000Z,37874853,@straits_times If one looks to the data of oth...,,


# Channel NewsAsia
## Get Tweets

In [11]:
CNA_tweets = getTweets('38400130', header)

for i in range(len(CNA_tweets['data'])):
    print('Tweet ID:', CNA_tweets['data'][i]['id'],
          'Time:', getTweetInformation(ST_tweets['data'][i]['id'], header))
    print(CNA_tweets['data'][i]['text'], '\n')
    
#     command = (
#             '''
#             INSERT INTO twitter_data
#             VALUES ('%s', '%s', '%s');
#             ''' % (CNA_tweets['data'][i]['id'], getTweetInformation(CNA_tweets['data'][i]['id'], header), 
#                    CNA_tweets['data'][i]['text'])
#             )
#     setUpDB(command, uri)

Tweet ID: 1492447657066840064 Time: 2022-02-12T10:37:05.000Z
Soccer - Klopp says Liverpool signings need to hit the ground running https://t.co/z57mpcYLQF https://t.co/MuKnFw4Jip 

Tweet ID: 1492445893630390273 Time: 2022-02-12T10:37:03.000Z
Recent rise in Singapore's COVID-19 cases 'not unexpected', says Ong Ye Kung https://t.co/gRtoKKAKip https://t.co/6u5cX1Apzx 

Tweet ID: 1492439724618752002 Time: 2022-02-12T10:23:32.000Z
Cross-country skiing-Russian Olympic Committee win gold in 4x5 km relay https://t.co/zdVr7uFp6F https://t.co/yIzwB5OmfX 

Tweet ID: 1492437724061581315 Time: 2022-02-12T10:11:37.000Z
Olympics - Figureskating-Skaters seek to shut out noise as world watches Valieva https://t.co/e1YIiooAjk https://t.co/jImh3X02tV 

Tweet ID: 1492437715253547008 Time: 2022-02-12T10:11:36.000Z
Hong Kong reports record COVID-19 cases, seeks China support https://t.co/bKAtKsHpLq https://t.co/EelXaOgml7 

Tweet ID: 1492430678385836035 Time: 2022-02-12T10:11:34.000Z
Speed skating-Russians 

## Get Conversation

In [12]:
CNA_conversation_data = getConversation('1492425387976327168', max_results, header)

result = getTweetComments(CNA_conversation_data)

User ID: 406301134 Time: 2022-02-12T10:29:26.000Z
In reply to: 38400130
@ChannelNewsAsia A pappy min saying cop is correct and proper when cop is led by a pappy min. 

User ID: 40854421 Time: 2022-02-12T10:26:11.000Z
In reply to: 38400130
@ChannelNewsAsia The message is if you want to join opposition, think twice, thrice.. 

User ID: 1412314720065916929 Time: 2022-02-12T10:08:42.000Z
In reply to: 38400130
@ChannelNewsAsia His boss and the Criminals Of PAP have  never stopped trying to fix .....

https://t.co/NKRvLAiWT5 

User ID: 2668705165 Time: 2022-02-12T09:27:32.000Z
In reply to: 38400130
@ChannelNewsAsia is he in the COP? 

User ID: 1368045292637933569 Time: 2022-02-12T09:27:29.000Z
In reply to: 38400130
@ChannelNewsAsia Want to make this apolitical? Every PAP candidate should recuse themselves from this investigation. Until that happens, stop this farce about WP politicising the issue. 

User ID: 2284351680 Time: 2022-02-12T09:18:31.000Z
In reply to: 38400130
@ChannelNewsAsia Suc

In [13]:
df = pd.DataFrame.from_dict(result)
df['id'] = df['id'].astype(str)
df['reply_to'] = df['reply_to'].astype(str)
df['url'] = df['comment'].apply(lambda x: getLinks(x))
df['link_title'] = df['url'].apply(lambda x: getURLfromList(x))
df.to_csv('Datasets/CNA_twitter_data.csv', index=False)

df.head()

,id,timestamp,reply_to,comment,url,link_title
0,406301134,2022-02-12T10:29:26.000Z,38400130,@ChannelNewsAsia A pappy min saying cop is cor...,,
1,40854421,2022-02-12T10:26:11.000Z,38400130,@ChannelNewsAsia The message is if you want to...,,
2,1412314720065916929,2022-02-12T10:08:42.000Z,38400130,@ChannelNewsAsia His boss and the Criminals Of...,https://www.youtube.com/watch?v=a1WhJKsYb50&fe...,[www.youtube.com]
3,2668705165,2022-02-12T09:27:32.000Z,38400130,@ChannelNewsAsia is he in the COP?,,
4,1368045292637933569,2022-02-12T09:27:29.000Z,38400130,@ChannelNewsAsia Want to make this apolitical?...,,
